In [1]:
import os 
from glob import glob 
import tensorflow as tf 
from tensorflow.keras.layers import Layer,Dropout,Dense,BatchNormalization,Flatten
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from openpose import *
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

[2022-04-08 15:00:28,167] [TfPoseEstimator] [INFO] loading graph from D:\Python Projects\Fall detection tf-pose\tf_pose_estimation\models\graph/cmu/graph_opt.pb(default size=432x368)


TfPoseEstimator/Openpose/concat_stage7/axis
TfPoseEstimator/Mconv7_stage6_L2/biases
TfPoseEstimator/Mconv7_stage6_L2/weights
TfPoseEstimator/Mconv6_stage6_L2/biases
TfPoseEstimator/Mconv6_stage6_L2/weights
TfPoseEstimator/Mconv5_stage6_L2/biases
TfPoseEstimator/Mconv5_stage6_L2/weights
TfPoseEstimator/Mconv4_stage6_L2/biases
TfPoseEstimator/Mconv4_stage6_L2/weights
TfPoseEstimator/Mconv3_stage6_L2/biases
TfPoseEstimator/Mconv3_stage6_L2/weights
TfPoseEstimator/Mconv2_stage6_L2/biases
TfPoseEstimator/Mconv2_stage6_L2/weights
TfPoseEstimator/Mconv1_stage6_L2/biases
TfPoseEstimator/Mconv1_stage6_L2/weights
TfPoseEstimator/Mconv7_stage6_L1/biases
TfPoseEstimator/Mconv7_stage6_L1/weights
TfPoseEstimator/Mconv6_stage6_L1/biases
TfPoseEstimator/Mconv6_stage6_L1/weights
TfPoseEstimator/Mconv5_stage6_L1/biases
TfPoseEstimator/Mconv5_stage6_L1/weights
TfPoseEstimator/Mconv4_stage6_L1/biases
TfPoseEstimator/Mconv4_stage6_L1/weights
TfPoseEstimator/Mconv3_stage6_L1/biases
TfPoseEstimator/Mconv3_st

In [2]:
import random
seed_constant = 50
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)
seq_len=20
w,h=432,368

In [3]:

data_set_dire='Dataset'
CLASSES_LIST = ["bend", "stand-up", "look around for something"]
seq_len=20
#resized_height,resized_width=128,128
num_features=36

In [4]:
def get_features(video_dire):
    extracted_features=[]
    video_reader=cv.VideoCapture(video_dire)
    total_frames=int(video_reader.get(cv.CAP_PROP_FRAME_COUNT))
    skip_frames=max(int(total_frames/seq_len),1)
    for frame_counter in range(total_frames):
        video_reader.set(cv.CAP_PROP_POS_FRAMES,frame_counter*skip_frames)
        sucess,frame=video_reader.read()
        if not sucess:
            break
        frame=cv2.resize(frame,(w,h))    
        feature,_=get_features_df(e,frame)
        extracted_features.append(feature)
        if len(extracted_features)==seq_len:
            break
    return np.array(extracted_features)

In [5]:
def create_dataset():
    labels=[]
    features=[]
    for idx,className in enumerate(CLASSES_LIST):
        video_dires_list=glob(os.path.join(data_set_dire,className)+"/**/*.avi")
        random.shuffle(video_dires_list)
        video_dires_list=video_dires_list[:700]
        for num_videes,video_dire in enumerate(video_dires_list):
            if num_videes %10==0:
                print("Num of Videos Processed : {}".format((idx+1)*num_videes))
            video_dire=video_dire.replace("\\","/")
            
            extracted_features= get_features(video_dire)
            
            
            features.append(extracted_features)
            labels.append(idx)
                
    return features,labels

In [6]:
features,labels=create_dataset()


Num of Videos Processed : 0
Num of Videos Processed : 10
Num of Videos Processed : 20
Num of Videos Processed : 30
Num of Videos Processed : 40
Num of Videos Processed : 50
Num of Videos Processed : 60
Num of Videos Processed : 70
Num of Videos Processed : 80
Num of Videos Processed : 90
Num of Videos Processed : 100
Num of Videos Processed : 110
Num of Videos Processed : 120
Num of Videos Processed : 130
Num of Videos Processed : 140
Num of Videos Processed : 150
Num of Videos Processed : 160
Num of Videos Processed : 170
Num of Videos Processed : 180
Num of Videos Processed : 190
Num of Videos Processed : 200
Num of Videos Processed : 210
Num of Videos Processed : 220
Num of Videos Processed : 230
Num of Videos Processed : 240
Num of Videos Processed : 250
Num of Videos Processed : 260
Num of Videos Processed : 270
Num of Videos Processed : 280
Num of Videos Processed : 290
Num of Videos Processed : 300
Num of Videos Processed : 310
Num of Videos Processed : 320
Num of Videos Process

In [7]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [8]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=42)

In [11]:
np.save("train_dataset/X_train",X_train)
np.save("train_dataset/X_test",X_test)
np.save("train_dataset/y_train",y_train)
np.save("train_dataset/y_test",y_test)